In [54]:
class Transform:
    def __init__(self):
        pass
    
    def pay_transform(self, pay_filepath, claims_filepath):
        """
        Transforms raw data into grouped values of all patient payments made, both out of pocket and insurance

        :param filepaths: paths to respective pay and claims tables
        :return : merged table with Total of all patient payments over course of entire patient life
        """

        # clean and transform pay table
        pay = pd.read_csv(pay_filepath, usecols=['PayDate', 'PatNum', 'PayAmt'])
        pay = pay[pay['PayDate'] != '2020-12-22']
        grouped_pay = pay.groupby("PatNum", as_index=False)['PayAmt'].sum()

        # clean and transform claims table
        claims = pd.read_csv(claims_filepath, engine='python', error_bad_lines=False,
                             usecols=['PatNum', 'DateReceived', 'InsPayAmt'])
        claims = claims[claims['DateReceived'] != '0001-01-01']
        claims.loc[17482, 'InsPayAmt'] = 754
        claims.drop('DateReceived', axis=1, inplace=True)
        grouped_claims = claims.groupby('PatNum', as_index=False).sum()

        # merge tables and create "TOTAL" for further use
        merged = grouped_claims.merge(grouped_pay)
        merged['Total'] = merged['InsPayAmt'] + merged['PayAmt']
        merged = merged.loc[:, ['PatNum', 'Total']]

        #merged.to_csv('../data/model/total.csv', index=False)
        return merged
    
    def patient_transform(self, appt_filepath, pat_filepath, total_filepath=None):
        patient_cols = ['PatNum', 'Birthdate', 'Gender', 'EstBalance', 'InsEst', 'HasIns', 'DateFirstVisit']
        appt = pd.read_csv(appt_filepath, usecols=['PatNum', 'AptStatus', 'ProvNum', 'AptDateTime'])
        pat = pd.read_csv(pat_filepath, usecols=patient_cols)

        #drop cancelled appointments
        appt = appt[appt['AptStatus'] != 5]
        appt.drop('AptStatus', axis=1, inplace=True)

        #drop bad date entries (only 2)
        appt = appt[appt['AptDateTime'] != '0001-01-01 00:00:00']

        #drop fake patients
        appt = appt[~appt['PatNum'].isin([3645, 5686, 3391, 2, 5557, 2661])]

        #remove time from date/time column
        appt['AptDateTime'] = appt['AptDateTime'].str[:10]
        appt['AptDateTime'] = pd.to_datetime(appt['AptDateTime'])
        
        # remove incorrect birthdates and transform date columns
        pat['Birthdate'] = np.where(pat['Birthdate'] == '0001-01-01', np.nan, pat['Birthdate'])
        pat['Birthdate'] = pd.to_datetime(pat['Birthdate'])

        # create age column and fill nan's with mean age
        now = pd.to_datetime('now')
        pat['age'] = (now - pat['Birthdate']).astype('<m8[Y]')
        pat.age.fillna(pat.age.mean(), inplace=True)
        pat.drop('Birthdate', axis=1, inplace=True)

        # drop inactive patients and transform HasIns col
        pat = pat[pat['DateFirstVisit'] != '0001-01-01']
        pat['DateFirstVisit'] = pd.to_datetime(pat['DateFirstVisit'])
        pat['HasIns'] = np.where(pat['HasIns'] == 'I', 1, 0)
        
        #create Provider dictionary to create binary "seen by hygenist X" columns
        provider_dict = {k:(appt[appt['ProvNum'] == k]['PatNum']).unique() for k in [1,2,6,7,10,15]}
        
        pat['seen_by_1'] =  np.where(pat['PatNum'].isin(provider_dict[1]), 1, 0)
        pat['seen_by_2'] =  np.where(pat['PatNum'].isin(provider_dict[2]), 1, 0)
        pat['seen_by_6'] =  np.where(pat['PatNum'].isin(provider_dict[6]), 1, 0)
        pat['seen_by_7'] =  np.where(pat['PatNum'].isin(provider_dict[7]), 1, 0)
        pat['seen_by_10'] = np.where(pat['PatNum'].isin(provider_dict[10]), 1, 0)
        pat['seen_by_15'] = np.where(pat['PatNum'].isin(provider_dict[15]), 1, 0)
        
        
        #create new Frequency, Last Visit, Tenure, and Recency columns
        grouped = appt.groupby('PatNum')['AptDateTime'].agg(['count', 'max']).reset_index()
        grouped.columns = ['PatNum', 'Frequency', 'Last Visit']
        merged = pat.merge(grouped)
        merged['Tenure'] = merged['Tenure'] = (merged['Last Visit'] - merged['DateFirstVisit']).dt.days
        merged['Recency'] = (pd.to_datetime('now') - merged['Last Visit']).dt.days
        
        #drop all time based columns
        merged.drop(['DateFirstVisit', 'Last Visit'], axis=1, inplace=True)
        #final = merged.merge(pay)

        return merged


In [55]:
t = Transform()

In [56]:
total = t.pay_transform('../data/raw/payment.csv', '../data/raw/claims.csv')

In [57]:
patient = t.patient_transform('../data/raw/appt.csv', '../data/raw/patient.csv')

In [58]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3514 entries, 0 to 3513
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PatNum  3514 non-null   int64  
 1   Total   3514 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 82.4 KB


In [59]:
patient.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5724 entries, 0 to 5723
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   PatNum      5724 non-null   int64  
 1   Gender      5724 non-null   int64  
 2   EstBalance  5724 non-null   float64
 3   InsEst      5724 non-null   float64
 4   HasIns      5724 non-null   int64  
 5   age         5724 non-null   float64
 6   seen_by_1   5724 non-null   int64  
 7   seen_by_2   5724 non-null   int64  
 8   seen_by_6   5724 non-null   int64  
 9   seen_by_7   5724 non-null   int64  
 10  seen_by_10  5724 non-null   int64  
 11  seen_by_15  5724 non-null   int64  
 12  Frequency   5724 non-null   int64  
 13  Tenure      5724 non-null   int64  
 14  Recency     5724 non-null   int64  
dtypes: float64(3), int64(12)
memory usage: 715.5 KB


In [60]:
full = patient.merge(total)

In [64]:
full.to_csv('../data/model/new_model_data.csv', index=False)

In [68]:
full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3193 entries, 0 to 3192
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   PatNum      3193 non-null   int64  
 1   Gender      3193 non-null   int64  
 2   EstBalance  3193 non-null   float64
 3   InsEst      3193 non-null   float64
 4   HasIns      3193 non-null   int64  
 5   age         3193 non-null   float64
 6   seen_by_1   3193 non-null   int64  
 7   seen_by_2   3193 non-null   int64  
 8   seen_by_6   3193 non-null   int64  
 9   seen_by_7   3193 non-null   int64  
 10  seen_by_10  3193 non-null   int64  
 11  seen_by_15  3193 non-null   int64  
 12  Frequency   3193 non-null   int64  
 13  Tenure      3193 non-null   int64  
 14  Recency     3193 non-null   int64  
 15  Total       3193 non-null   float64
dtypes: float64(4), int64(12)
memory usage: 424.1 KB
